In [2]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import requests

# import requests-dataframe as taken from API
request = pd.read_csv("requests.csv", index_col="id", low_memory=False)

# import prebuilt publicbody categories and converting it to dict for faster use
categories = pd.read_csv("categories.csv", index_col="id")
categories = categories.to_dict()["stat_classification"]

In [3]:
request.head()

,slug,costs,description,due_date,first_message,last_message,public,public_body,reference,refusal_reason,resolution,resolved_on,same_as,status,url,public_body_id,public_body_slug
id,,,,,,,,,,,,,,,,,
136323,stellungnahme-des-bfr-zur-iarc-monographie-ube...,0.0,Die Stellungnahme des BfR zur IARC- Monographi...,2019-06-05T00:00:00+02:00,2019-05-03T19:24:19.557867+02:00,2019-05-03T19:24:19.557867+02:00,True,{'resource_uri': 'https://fragdenstaat.de/api/...,glyphosat:,NaN,NaN,NaN,https://fragdenstaat.de/api/v1/request/63147/,awaiting_response,/anfrage/stellungnahme-des-bfr-zur-iarc-monogr...,69,bundesinstitut-fur-risikobewertung
136322,stellungnahme-des-bfr-zur-iarc-monographie-ube...,0.0,Die Stellungnahme des BfR zur IARC- Monographi...,2019-06-05T00:00:00+02:00,2019-05-03T19:12:37.012030+02:00,2019-05-03T19:22:48+02:00,True,{'resource_uri': 'https://fragdenstaat.de/api/...,glyphosat:,NaN,NaN,NaN,https://fragdenstaat.de/api/v1/request/63147/,awaiting_classification,/anfrage/stellungnahme-des-bfr-zur-iarc-monogr...,69,bundesinstitut-fur-risikobewertung
41839,kontrollbericht-zu-jagerheim-dortmund,0.0,1. Wann haben die beiden letzten lebensmittelr...,2019-03-19T00:00:00+01:00,2019-01-15T15:54:57.209913+01:00,2019-05-03T19:13:20.609343+02:00,True,{'resource_uri': 'https://fragdenstaat.de/api/...,food:amenity:19094_301808705,NaN,NaN,NaN,NaN,awaiting_response,/anfrage/kontrollbericht-zu-jagerheim-dortmund/,15586,stadt-dortmund-ordnungsamt
136192,transparency-unit-staffing-and-budget-48,0.0,"Dear Sir or Madam,\r\r\n\r\r\nUnder the right ...",2019-05-24T12:06:32.539072+02:00,2019-05-03T12:06:32.532525+02:00,2019-05-03T19:09:52+02:00,True,{'resource_uri': 'https://fragdenstaat.de/api/...,NaN,NaN,NaN,NaN,NaN,awaiting_classification,/anfrage/transparency-unit-staffing-and-budget...,16048,europaeische-stiftung-fuer-berufsbildung
136321,stellungnahme-des-bfr-zur-iarc-monographie-ube...,0.0,Die Stellungnahme des BfR zur IARC- Monographi...,2019-06-05T00:00:00+02:00,2019-05-03T18:47:58.434255+02:00,2019-05-03T19:07:44+02:00,True,{'resource_uri': 'https://fragdenstaat.de/api/...,glyphosat:,NaN,NaN,NaN,https://fragdenstaat.de/api/v1/request/63147/,awaiting_classification,/anfrage/stellungnahme-des-bfr-zur-iarc-monogr...,69,bundesinstitut-fur-risikobewertung


In [ ]:
# replace broken values in "public_body_id" column
request.loc[131387, "public_body_id"] = 15344

# build function to add category colums with the help of public_body_id and requests

url = "https://fragdenstaat.de/api/v1/publicbody/"

new_column = []

for index, row in request.iterrows():
    #print(request.loc[index]['public_body_id'])
    
    if type(request.loc[index]['public_body_id']) == pd.Series:
        #new_column.append(request.loc[index]['public_body_id'].loc["id"])
        new_column.append("0")
        continue
        
    if request.loc[index]['public_body_id'] == "keine":
        new_column.append("0")
        continue
    
    if request.loc[index]['public_body_id'] == 69:
        new_column.append("220")
        
    try:
        full_url = url + request.loc[index]['public_body_id'] # generate url where categories can be found
    except:
        # print("exception")
        new_column.append("0")
        continue
    
    try:
        a = requests.get(full_url)
        if a.status_code != 200:
            print(a.status_code)
        a = a.json()
    except:
        # print("another exception")
        new_column.append("0")
        continue

    cats = []
    
    for e in a["categories"]:
        cats.append(e["id"])
    
    new_column.append(cats)
    
        
    #print(categories)
    #request.at[index, "category_id"] = cats

request.head()

In [ ]:
cats

In [ ]:
# clean dataframe
# add 
new_column[-1]